## Constant Tensors

In [1]:
import tensorflow as tf

In [3]:
x = tf.constant([[5, 2], [1, 3]])
print(x)

tf.Tensor(
[[5 2]
 [1 3]], shape=(2, 2), dtype=int32)


In [4]:
x.numpy()

array([[5, 2],
       [1, 3]], dtype=int32)

In [5]:
x.dtype

tf.int32

In [7]:
x.shape

TensorShape([2, 2])

In [8]:
print(tf.ones(shape=(2, 1)))
print(tf.zeros(shape=(2, 1)))

tf.Tensor(
[[1.]
 [1.]], shape=(2, 1), dtype=float32)
tf.Tensor(
[[0.]
 [0.]], shape=(2, 1), dtype=float32)


In [10]:
x = tf.random.normal(shape=(2, 2), mean=0.0, stddev=1.0)
x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.57473147,  0.03271284],
       [-0.02064646, -0.55824697]], dtype=float32)>

In [11]:
x = tf.random.uniform(shape=(2, 2), minval=0, maxval=10, dtype="int32")
x

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[6, 7],
       [6, 4]], dtype=int32)>

## Variable

In [13]:
a = tf.Variable(5)
a

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=5>

In [14]:
a.assign(10)
a

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=10>

In [15]:
a.assign_add(3)
a

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=13>

In [17]:
a = tf.random.normal(shape=(2, 2))
b = tf.random.normal(shape=(2, 2))
print(a)
print(b)

tf.Tensor(
[[-0.13878174  2.0602763 ]
 [ 1.6470094   1.2440869 ]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[ 0.02803783 -0.09722872]
 [-1.0603919   1.0085369 ]], shape=(2, 2), dtype=float32)


In [18]:
c = a + b
c

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-0.11074392,  1.9630475 ],
       [ 0.58661747,  2.2526238 ]], dtype=float32)>

In [19]:
d = tf.square(c)
d

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[0.01226422, 3.8535554 ],
       [0.34412006, 5.074314  ]], dtype=float32)>

In [20]:
e = tf.exp(d)
e

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[  1.0123397,  47.160442 ],
       [  1.410748 , 159.8625   ]], dtype=float32)>

In [22]:
tf.matmul(a, b)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.1885915,  2.0913582],
       [-1.273041 ,  1.094571 ]], dtype=float32)>

In [51]:
d = tf.constant([[-2, 6, 12, 0 , -5], [12, -5, 3, 2 , 0]], dtype='float32')

In [52]:
tf.nn.relu(d)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[ 0.,  6., 12.,  0.,  0.],
       [12.,  0.,  3.,  2.,  0.]], dtype=float32)>

In [53]:
tf.nn.softmax(d)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[8.2946690e-07, 2.4726060e-03, 9.9752039e-01, 6.1289775e-06,
        4.1296726e-08],
       [9.9982506e-01, 4.1392134e-08, 1.2338821e-04, 4.5391986e-05,
        6.1431374e-06]], dtype=float32)>

## Model subclassing to create a new Layer

In [23]:
from tensorflow import keras

class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=100, input_dim=784):
        super().__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


In [24]:
# Instantiate our layer.
linear_layer = Linear(units=100, input_dim=25)

# The layer can be treated as a function.
# Here we call it on some data.
y = linear_layer(tf.ones((10, 25)))
assert y.shape == (10, 100)

In [25]:
linear_layer.weights

[<tf.Variable 'Variable:0' shape=(25, 100) dtype=float32, numpy=
 array([[-0.03194964, -0.04348142,  0.01325685, ...,  0.02421599,
         -0.0515284 , -0.06371283],
        [ 0.03563127, -0.01206199,  0.06074193, ..., -0.06958984,
          0.07691862,  0.07958107],
        [-0.05514812,  0.00185488, -0.02547662, ...,  0.00122426,
         -0.01371243, -0.04202339],
        ...,
        [-0.04197098, -0.03454831, -0.05753744, ..., -0.04240061,
         -0.12929021,  0.00185433],
        [-0.0619064 ,  0.02643344,  0.05203336, ..., -0.02812349,
         -0.04306407, -0.05824762],
        [ 0.0328081 ,  0.02998362,  0.03904473, ..., -0.00817718,
          0.09826884, -0.01553315]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(100,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
  

In [26]:
len(linear_layer.weights)

2

In [27]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

### lets create a sequential Model with out custom Layer:

In [35]:
model = keras.Sequential()
model.add(Linear(100, 784))
model.add(keras.layers.Activation('relu'))
model.add(Linear(200, 100))
model.add(keras.layers.Activation('relu'))
model.add(Linear(10, 200))
model.add(keras.layers.Activation('Softmax'))

In [36]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

In [37]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
1875/1875 [==============================] - 11s 4ms/step - loss: 0.2595 - accuracy: 0.9235 - val_loss: 0.1308 - val_accuracy: 0.9602
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1074 - accuracy: 0.9669 - val_loss: 0.0930 - val_accuracy: 0.9708
Epoch 3/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0736 - accuracy: 0.9772 - val_loss: 0.0841 - val_accuracy: 0.9747
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0565 - accuracy: 0.9821 - val_loss: 0.0810 - val_accuracy: 0.9751
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0442 - accuracy: 0.9852 - val_loss: 0.0828 - val_accuracy: 0.9754


### do not repeat input_dim!
#### Layer weight creation in build(input_shape)


In [38]:
class Linear(keras.layers.Layer):
    """y = w.x + b"""

    def __init__(self, units=32):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


# Instantiate our layer.
linear_layer = Linear(4)

# This will also call `build(input_shape)` and create the weights.
y = linear_layer(tf.ones((10, 784)))


In [40]:
model = keras.Sequential()
model.add(keras.Input(784,))
model.add(Linear(100))
model.add(keras.layers.Activation('relu'))
model.add(Linear(200))
model.add(keras.layers.Activation('relu'))
model.add(Linear(10))
model.add(keras.layers.Activation('Softmax'))

In [41]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Epoch 1/2
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2589 - accuracy: 0.9236 - val_loss: 0.1196 - val_accuracy: 0.9623
Epoch 2/2
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1050 - accuracy: 0.9677 - val_loss: 0.1097 - val_accuracy: 0.9658


### Lets give activation as an Input

In [58]:
class FC(keras.layers.Layer):
    """y = Activation(w.x + b)"""

    def __init__(self, units=32, activation='linear'):
        super().__init__()
        self.units = units
        if activation=='linear':
          self.activation = lambda x:x
        elif activation=='relu':
          self.activation = tf.nn.relu
        elif activation=='tanh':
          self.activation = tf.nn.tanh
        elif activation=='sigmoid':
          self.activation = tf.nn.sigmoid
        elif activation=='softmax':
          self.activation = tf.nn.softmax

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return self.activation (tf.matmul(inputs, self.w) + self.b)


# Instantiate our layer.
linear_layer = FC(25)

# This will also call `build(input_shape)` and create the weights.
y = linear_layer(tf.ones((10, 784)))


In [60]:
model = keras.Sequential()
model.add(keras.Input(784,))
model.add(FC(100, activation='relu'))
model.add(FC(200, activation='relu'))
model.add(FC(10, activation='softmax'))

In [61]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Epoch 1/2
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2507 - accuracy: 0.9266 - val_loss: 0.1216 - val_accuracy: 0.9617
Epoch 2/2
1875/1875 [==============================] - 7s 4ms/step - loss: 0.1047 - accuracy: 0.9675 - val_loss: 0.0979 - val_accuracy: 0.9702
